In [1]:
import numpy as np
import pandas as pd
from numpy import exp, array
from hw import Jamshidian as jamsh
from hw import Henrard as henr
from hw import calibration as hw_calib
import hw.calibration as calib
from hw.const import *
from fox_toolbox.utils import xml_parser as log
from fox_toolbox.utils import rates
from scipy.stats import norm
import scipy.integrate as integrate
from numpy import exp, sqrt
from scipy.stats import norm
from fox_toolbox.utils import volatility as vols

%load_ext autoreload
%autoreload 2

### Read IRSM FORM

In [2]:
_, irsmform = log.get_xml(INPUT_1SWO)
main_curve, sprds = log.get_curves(irsmform)
dsc_curve = main_curve

try:
    estim_curve = sprds[0]
except TypeError:
    estim_curve = main_curve

cal_basket = list(log.get_calib_basket(irsmform))
swo = cal_basket[0]

In [3]:
fwd = swo.get_swap_rate(main_curve, estim_curve)
flt_adjs = swo.get_flt_adjustments(main_curve, estim_curve)

calib_annuity = swo.get_annuity(main_curve)

### READ IRSM OUT

In [4]:
_, irsmout = log.get_xml(OUTPUT_1SWO)
ref_swo = list(log.get_calib_basket(irsmout))[0]

# ref_swos = list(log.get_calib_basket(irsmout))

ref_mr, (hw_buckets, hw_sigma) = log.get_hw_params(irsmout)

ref_sigmas = rates.Curve1d(hw_buckets, hw_sigma, 'PieceWise')

In [5]:
ref_sigma = ref_sigmas.values[0]

### MAIN HW (Jamshidian)

In [6]:
coefJ = jamsh.get_coef(swo, ref_mr, ref_sigma, main_curve, estim_curve)
b_i = jamsh.get_b_i(swo, ref_mr)
varx = jamsh.get_var_x(swo.expiry, ref_mr, ref_sigma)

x_starJ = jamsh.get_x_star(coefJ, b_i, varx)

jamsh_price = jamsh.hw_swo(swo, ref_mr, ref_sigma, main_curve, estim_curve) #* main_curve.get_dsc(swo.expiry)
jamsh_price

0.013734203599531935

## MAIN HW (Hernard)

In [ ]:
start_date = swo.start_date

hw_dates = np.insert(swo.payment_dates, 0, start_date)

coefH = henr.get_coef(swo.day_count_fractions, swo.strike, flt_adjs)
P_i = henr.get_P_i(dsc_curve, hw_dates)
alpha_i, debug_total_var = henr.get_alpha_i(ref_mr, swo.expiry, hw_dates, ref_sigmas)

x_starH = henr.get_x_star(coefH, P_i, alpha_i)

henr_price, debug, debug_total_var = henr.hw_swo(swo, ref_mr, ref_sigmas, dsc_curve, estim_curve)
debug_df = pd.DataFrame(data=debug)
henr_price

In [13]:
debug_df

,Expiry,hw_dates,coef,P_i,alpha_i,x_star,IsAnalytic,degen_coef,degen_x_star,correction
0,0.99726,1.005479,-1.000000,0.998545,0.000016,NaN,False,-1.000000,-1.529294,1.099441e-09
1,0.99726,2.010959,0.003373,0.997517,0.001887,NaN,False,0.003373,-1.529294,1.099441e-09
2,0.99726,3.008219,0.000112,0.995364,0.003653,NaN,False,0.000112,-1.529294,1.099441e-09
3,0.99726,4.008219,-0.003203,0.993209,0.005337,NaN,False,0.000000,-1.529294,1.099441e-09
4,0.99726,5.008219,0.993485,0.991059,0.006939,NaN,False,0.993485,-1.529294,1.099441e-09


In [14]:
debug_total_var

Curve1d('HW total var by expiries', array([0.        , 0.99726027]), array([0.00000000e+00, 4.02798435e-06]), PieceWise)

### Bachelier price from market info

In [11]:
w = -1.0 if swo.pay_rec == 'Receiver' else 1.0

black_price = calib_annuity * vols.BachelierPrice(fwd, swo.strike, swo.vol.value * np.sqrt(swo.expiry), w=w)
black_price

0.013706940103996023

## HW Calibration

In [23]:
sigma_hw_jamsh = hw_calib.calibrate_sigma_hw(cal_basket, ref_mr, dsc_curve, estim_curve, True)
100*sigma_hw_jamsh

0.9972602739726
Curve1d
--------------------------------------------------------------------------------
Name: 'HW model sigma'
Pillars: array([0.])
Zero-coupons: array([nan])
Interpolation: 'PieceWise'


TypeError: unsupported operand type(s) for -: 'NoneType' and 'NoneType'

In [31]:
sigma_hw_henr, calib_debug = hw_calib.calibrate_sigma_hw(cal_basket, ref_mr, dsc_curve, estim_curve, False)
print(sigma_hw_henr)

Curve1d
--------------------------------------------------------------------------------
Name: 'HW model sigma'
Pillars: array([ 0.        ,  0.99726027, 30.99726027])
Zero-coupons: array([       nan, 0.00195392, 0.00195392])
Interpolation: 'PieceWise'


In [32]:
print(ref_sigmas)

Curve1d
--------------------------------------------------------------------------------
Name: ''
Pillars: array([ 0.        ,  0.99726027, 30.99726027])
Zero-coupons: array([0.00195985, 0.00195985, 0.00195985])
Interpolation: 'PieceWise'


In [22]:
100*(sigma_hw_henr - ref_sigma)/ref_sigma

-0.30219544432282647

In [39]:
print('fwd recon :', ref_swo.fwd - fwd)
print('Annuity recon', ref_swo.annuity - calib_annuity)
print('')
print('***Jamshidian pricing info***')
print('x_star Jamshidian', x_starJ)
print('swap_value Jamshidian', jamsh.swap_value(coefJ, b_i, varx, x_starJ))
print('')
print('***Henrard pricing info***')
print('x_star Hernard', x_starH)
print('swap_value Henrard', henr.swap_value(coefH, P_i, alpha_i, x_starH))
print(' ')
print('TEST Bachelier -> Black price recon', black_price - ref_swo.tgt_premium )
print(' ')
print('***Prices %***')
print('Jamsh price with <RefSigma>         ', 100*jamsh_price)
print('Hern price with <RefSigma>          ', 100*henr_price)
print('<BlackPrice>                        ', 100*ref_swo.tgt_premium)
print('<CalibPremium>                      ', 100*ref_swo.cal_premium)
print('Jamsh price with my Jamsh sigma     ', 100*jamsh.hw_swo(swo, ref_mr, sigma_hw_jamsh, main_curve, estim_curve))
print('Hern price with my hern sigma       ', 100*henr.hw_swo(swo, ref_mr, sigma_hw_henr, main_curve, estim_curve))
print(' ')
print('***Calibration: sigma Hull White %***')
print('My calibrated Jamshidian sigma:     ', 100*sigma_hw_jamsh)
print('My calibrated Henrard sigma:        ', 100*sigma_hw_henr)
print('<RefSigma> (Henrard):               ', 100*ref_sigma)

fwd recon : 7.23349331821499e-14
Annuity recon -6.128431095930864e-14

***Jamshidian pricing info***
x_star Jamshidian -1.995775631165543
swap_value Jamshidian -4.3298697960381105e-15

***Henrard pricing info***
x_star Hernard -1.9948237621088238
swap_value Henrard -1.1102230246251565e-16
 
TEST Bachelier -> Black price recon -3.039773294188919e-13
 
***Prices %***
Jamsh price with <RefSigma>          1.3734203599531936
Hern price with <RefSigma>           1.3708071029350788
<BlackPrice>                         1.37069401043
<CalibPremium>                       1.37069398165
Jamsh price with my Jamsh sigma      1.3706940103819307
Hern price with my hern sigma        1.3706940103995868
 
***Calibration: sigma Hull White %***
My calibrated Jamshidian sigma:      0.1797215337202844
My calibrated Henrard sigma:         0.19539238377504808
<RefSigma> (Henrard):                0.19598464043


#### ! <font color='green'>got better calibration than logs </font> !

In [ ]:
fwd recon : 7.23349331821499e-14
Annuity recon -6.128431095930864e-14

***Jamshidian pricing info***
x_star Jamshidian -1.995775631165543
swap_value Jamshidian -4.3298697960381105e-15

***Henrard pricing info***
x_star Hernard -1.9948237621088238
swap_value Henrard -1.1102230246251565e-16
 
TEST Bachelier -> Black price recon -3.039773294188919e-13
 
***Prices %***
Jamsh price with <RefSigma>          1.3734203599531936
Hern price with <RefSigma>           1.3708071029350788
<BlackPrice>                         1.37069401043
<CalibPremium>                       1.37069398165
Jamsh price with my Jamsh sigma      1.3706940103819307
Hern price with my hern sigma        1.3706940103995868
 
***Calibration: sigma Hull White %***
My calibrated Jamshidian sigma:      0.1797215337202844
My calibrated Henrard sigma:         0.19539238377504808
<RefSigma>:                          0.19598464043